# Test Data Maker

This is a simple application to manually label test data for our usage. Running below will show a text and ask if the given sentence contains the information on target symptom or not. It will automatically start off from where we left off until all descriptions have been assessed. 

In [25]:
import pandas as pd
import joblib

### Processing dataset
Run this only once.

In [3]:
## transcription data
#mt = pd.read_csv('data/medical_transcription_samples.csv', index_col = 0)

In [7]:
## get a list of descriptions
#descriptions = pd.concat([mt['description'], mt['transcription']])

In [14]:
## get a list of target symptoms
#symptoms = pd.read_json('data/symptoms.json', orient = 'table')
#symptoms = symptoms['name']

In [23]:
## creating full dataframe
#df = pd.concat([descriptions,pd.DataFrame(columns=list(symptoms) + ['complete'])])
#df.rename(columns = {0: 'description'}, inplace = True)

In [26]:
## saving the dataframe
#joblib.dump(df, 'data/test_df')

['data/test_df']

### Labeling App

In [27]:
df = joblib.load('data/test_df')

In [ ]:
def labeling_app(df):
    # find incomplete rows
    
    # randomize the rownumber
    
    # find incomplete columns
    
    # randomize the columns
    
    # show the description
    
    # initialize the last seen variable
    
    # loop through the columns 
    
        # get yes/no answer for each column
        
        # if yes or no, update the last seen
        # and update the dataframe 
        
        # if prompted to go back, bring up the last option again

        # if prompted to finish, save the current dataframe and return the dataframe
        
        